In [572]:
# imports

import numpy as np 
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

In [573]:
# reading in the dataset
data = pd.read_csv("/kaggle/input/star-dataset/6 class csv.csv")

In [574]:
data.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [575]:
data.tail()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
235,38940,374830.0,1356.0,-9.93,5,Blue,O
236,30839,834042.0,1194.0,-10.63,5,Blue,O
237,8829,537493.0,1423.0,-10.73,5,White,A
238,9235,404940.0,1112.0,-11.23,5,White,A
239,37882,294903.0,1783.0,-7.80,5,Blue,O


In [576]:
data.shape

(240, 7)

In [577]:
data.dtypes

Temperature (K)             int64
Luminosity(L/Lo)          float64
Radius(R/Ro)              float64
Absolute magnitude(Mv)    float64
Star type                   int64
Star color                 object
Spectral Class             object
dtype: object

In [578]:
# setting prediction value and determining unique categorical features to rename columns later
predict = "Star type"

print(data['Star color'].unique())
print(data['Spectral Class'].unique())

['Red' 'Blue White' 'White' 'Yellowish White' 'Blue white'
 'Pale yellow orange' 'Blue' 'Blue-white' 'Whitish' 'yellow-white'
 'Orange' 'White-Yellow' 'white' 'Blue ' 'yellowish' 'Yellowish'
 'Orange-Red' 'Blue white ' 'Blue-White']
['M' 'B' 'A' 'F' 'O' 'K' 'G']


In [579]:
data_copy = data

In [580]:
#One Hot Encoding categorical data for model

encoder = OneHotEncoder(handle_unknown='ignore')
encoder_df = pd.DataFrame(encoder.fit_transform(data[['Star color', 'Spectral Class']]).toarray())
data = data.join(encoder_df)

In [581]:
# Dropping and renaming column data titles
data.drop('Star color',axis=1, inplace=True)
data.drop('Spectral Class',axis=1, inplace=True)
data.columns = ['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)', 'Absolute magnitude(Mv)', 'Star type', 'Red', 'Blue White', 'White', 'Yellowish White', 'Blue white',
       'Pale yellow orange', 'Blue', 'Blue-white', 'Whitish',
       'yellow-white', 'Orange', 'White-Yellow', 'white', 'Blue ',
       'yellowish', 'Yellowish', 'Orange-Red', 'Blue white ',
       'Blue-White', 'M', 'B', 'A', 'F', 'O', 'K', 'G']

In [582]:
data_copy.drop('Star color',axis=1, inplace=True)
data_copy.drop('Spectral Class',axis=1, inplace=True)
data_copy.drop('Temperature (K)',axis=1, inplace=True)

In [583]:
data.corr(method='pearson')

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Red,Blue White,White,Yellowish White,Blue white,...,Orange-Red,Blue white,Blue-White,M,B,A,F,O,K,G
Temperature (K),1.000000,0.393404,0.064216,-0.420261,0.411129,0.655403,0.063531,0.090590,-0.004274,0.009517,...,0.023171,-0.078529,-0.059584,-0.020150,0.463676,-0.057368,-0.024751,-0.100755,-0.704541,0.553433
Luminosity(L/Lo),0.393404,1.000000,0.526516,-0.692619,0.676845,0.448639,-0.038722,-0.124822,-0.067350,-0.038722,...,-0.067350,-0.111159,-0.054876,-0.093876,-0.078889,-0.165280,0.044004,0.040074,-0.237086,0.557902
Radius(R/Ro),0.064216,0.526516,1.000000,-0.608728,0.660975,-0.019493,-0.029723,-0.095816,-0.051700,-0.029724,...,-0.051700,-0.084917,-0.041978,-0.057542,-0.033152,-0.126585,0.154147,0.073935,0.066032,0.017883
Absolute magnitude(Mv),-0.420261,-0.692619,-0.608728,1.000000,-0.955276,-0.363879,0.042819,0.153523,0.102594,0.043065,...,0.094530,-0.018174,0.017309,-0.008291,-0.030567,0.111102,-0.088944,-0.062693,0.351736,-0.467128
Star type,0.411129,0.676845,0.660975,-0.955276,1.000000,0.351192,-0.018938,-0.061047,-0.032939,-0.018938,...,-0.032939,0.054366,0.026838,0.058735,0.092975,-0.004755,0.094689,0.109388,-0.447735,0.399339
Red,0.655403,0.448639,-0.019493,-0.363879,0.351192,1.000000,-0.035269,-0.113692,-0.061345,-0.035269,...,-0.061345,-0.101250,-0.049983,-0.159873,0.112289,-0.150545,-0.035269,-0.087310,-0.505781,0.820200
Blue White,0.063531,-0.038722,-0.029723,0.042819,-0.018938,-0.035269,1.000000,-0.013488,-0.007278,-0.004184,...,-0.007278,-0.012012,-0.005930,-0.018966,0.132838,-0.017860,-0.004184,-0.010358,-0.060002,-0.028928
White,0.090590,-0.124822,-0.095816,0.153523,-0.061047,-0.113692,-0.013488,1.000000,-0.023460,-0.013488,...,-0.023460,-0.038720,-0.019114,0.016089,0.375237,-0.057572,-0.013488,-0.033389,-0.193421,-0.093250
Yellowish White,-0.004274,-0.067350,-0.051700,0.102594,-0.032939,-0.061345,-0.007278,-0.023460,1.000000,-0.007278,...,-0.012658,-0.020892,-0.010314,0.244812,0.040494,-0.031064,-0.007278,-0.018016,-0.104365,-0.050315
Blue white,0.009517,-0.038722,-0.029724,0.043065,-0.018938,-0.035269,-0.004184,-0.013488,-0.007278,1.000000,...,-0.007278,-0.012012,-0.005930,-0.018966,0.132838,-0.017860,-0.004184,-0.010358,-0.060002,-0.028928


In [584]:
# Setting X and y arrays for linear regression and KNN, with y prediction value being the star type

X = np.array(data.drop([predict], 1))
y = np.array(data[predict])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [585]:
# Setting X and y arrays with only Luminosity, Radius, and Magnitude


X1 = np.array(data_copy.drop([predict], 1))
y1 = np.array(data_copy[predict])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [586]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X1, y1, test_size = 0.2)

linear = linear_model.LinearRegression()
linear.fit(x_train, y_train)

accuracy = linear.score(x_test, y_test)
print('Linear regression prediction accuracy with Luminosity, Radius, and Magnitude as features: ',accuracy * 100,'%')

Linear regression prediction accuracy with Luminosity, Radius, and Magnitude as features:  92.4283912084837 %


In [587]:
# Training and testing linear regression model a

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.2)

linear = linear_model.LinearRegression()
linear.fit(x_train, y_train)

accuracy = linear.score(x_test, y_test)
print('Linear regression prediction accuracy with all features: ',accuracy * 100,'%')

Linear regression prediction accuracy with all features:  97.59700057959135 %


In [588]:
x2_train, x2_test, y2_train, y2_test = sklearn.model_selection.train_test_split(X1, y1, test_size=0.2) 

neighbors = KNeighborsClassifier(n_neighbors=7)
neighbors.fit(x2_train, y2_train)
acc = neighbors.score(x2_test, y2_test)
print('KNN prediction accuracy with Luminosity, Radius, and Magnitude as featuers: ',acc * 100,'%')

KNN prediction accuracy with Luminosity, Radius, and Magnitude as featuers:  52.083333333333336 %


In [589]:
x2_train, x2_test, y2_train, y2_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2) 

neighbors = KNeighborsClassifier(n_neighbors=7)
neighbors.fit(x2_train, y2_train)
acc = neighbors.score(x2_test, y2_test)
print('KNN prediction accuracy with all features: ',acc * 100,'%')

KNN prediction accuracy with all features:  62.5 %
